2-Qubit Bloch Simulator — Guided Tutorial

This notebook demonstrates how to:

Build a simple 2-qubit circuit,

Simulate continuous evolution (including non-local gates like CX),

Apply discretized Kraus noise on selected qubits,

Track reduced states on each qubit (Bloch vectors), and

Compute global purity, local von Neumann entropies, and concurrence.

Run cells top to bottom.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 120

from IPython.display import HTML

from qiskit.quantum_info import Statevector, DensityMatrix
from bloch_simulator import (
    H1, I1, CX,
    embed2, total_noise_steps,
    discreet_depolarizing, discreet_T1, discreet_T2,
    continuous_path_2q, entropy_vN, concurrence_2q,
    radius, plot_purity, plot_entropy_lists, plot_concurrence,
    animate_trajectory
)


Initialize a 2-qubit basis state

We start from |00⟩ and convert it to a DensityMatrix

In [ ]:
ket00 = Statevector([1, 0, 0, 0])
rho00 = DensityMatrix(ket00)
rho00


Define a minimal entangling sequence

We apply (I ⊗ H) → CX → (I ⊗ H) and discretize each gate for continuous evolution.

In [ ]:
seq_A = [H1]    # applied on the second qubit via embedding
seq_B = [I1]    # identity on the first qubit
seq   = embed2(seq_B, seq_A)
seq.append(CX)
seq += embed2(seq_B, seq_A)

steps_per_gate = 90
dt = 0.001
n_seq = len(seq)
n_seq


Simulate noiseless evolution and record reduced states

The simulator returns Bloch vectors for qubit A and B at each sub-step, plus the global purity.
We also keep the full ρ(t) history to compute the concurrence.

In [ ]:
pts0_A, pts0_B, purity0, rhos0 = continuous_path_2q(
    rho00, seq, steps_per_gate=steps_per_gate, return_rhos=True
)
len(pts0_A), len(pts0_B), len(purity0), len(rhos0)


Add noise on one qubit (example: T1 on qubit A)

We lift 1-qubit Kraus operators to 4×4 using the project’s utilities.

In [ ]:
t_noise_steps = total_noise_steps(0, n_seq, steps_per_gate=steps_per_gate, n_ops=n_seq)
t_noise = t_noise_steps * dt

kraus_T1_A = discreet_T1(t_noise, 0.4, total_steps=t_noise_steps)  # on A
kraus_I_B  = [[I1, I1]]                                               # identity on B (shape matching)

# Lift to 4x4 for each step (same shape as your example)
import numpy as np
kraus_channels = [
    [k for k in (np.kron(a, b) for a, b in zip(kraus_T1_A, kraus_I_B[0]))]
]

pts_A, pts_B, purity, rhos = continuous_path_2q(
    rho00, seq, steps_per_gate=steps_per_gate,
    kraus_channels=kraus_channels,
    noise_start=0, noise_end=n_seq,
    return_rhos=True
)
len(pts_A), len(pts_B), len(purity), len(rhos)


Physical metrics

Radius on each qubit (noisy vs. noiseless),

Global purity over time,

Local von Neumann entropies,

Wootters concurrence from ρ(t).

In [ ]:
# Radius vs. time (noiseless and noisy)
radius(pts0_A, dt, title="A radius without noise")
radius(pts0_B, dt, title="B radius without noise")
radius(pts_A,  dt, title="A radius with noise")
radius(pts_B,  dt, title="B radius with noise")

# Purity
plot_purity(purity0, dt, title="System purity without noise")
plot_purity(purity,  dt, title="System purity with noise")

# Entropies (local)
S0_A, S0_B = entropy_vN(pts0_A), entropy_vN(pts0_B)
S_A,  S_B  = entropy_vN(pts_A),  entropy_vN(pts_B)
plot_entropy_lists(S0_A, dt, title="S(ρ_A) without noise")
plot_entropy_lists(S0_B, dt, title="S(ρ_B) without noise")
plot_entropy_lists(S_A,  dt, title="S(ρ_A) with noise")
plot_entropy_lists(S_B,  dt, title="S(ρ_B) with noise")

# Concurrence
C0 = [concurrence_2q(r) for r in rhos0]
C  = [concurrence_2q(r) for r in rhos]
plot_concurrence(C0, dt, title="Concurrence C(t) without noise")
plot_concurrence(C,  dt, title="Concurrence C(t) with noise")


Inline animations for both qubits

We animate each reduced Bloch trajectory (no external writers required).

In [ ]:
anim0_A, anim0_B = animate_trajectory(pts0_A, pts0_B, interval_ms=20)
anim_A,  anim_B  = animate_trajectory(pts_A,  pts_B,  interval_ms=20)

HTML(anim0_A.to_jshtml())
HTML(anim0_B.to_jshtml())
HTML(anim_A.to_jshtml())
HTML(anim_B.to_jshtml())
